In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

# for aesthetics/plotting
sns.set_theme()
plt.style.use("seaborn-v0_8")

In [3]:
from preprocess import AnomalyDetectionDataset, device_change
from model import AutoencoderAnomalyDetection, train_loop, test_loop, train_model
from torch import nn

In [4]:
data_dir = '/users/eserebri/data/eserebri/PHYS2550FinalProject/anomaly detection'
train_path = f'{data_dir}/X_train_single_ch.csv'
test_path = f'{data_dir}/X_test_single_ch.csv'
valid_path = f'{data_dir}/X_valid_ch.csv'

### Getting Data with pytorch

In [5]:
device = device_change()

Using mps device


In [11]:
# background = AnomalyDetectionDataset(filepath_background, three_channels=True)
X_train = ...
X_test = ...
X_valid = ...

### Model Implementation

In [7]:
learning_rate = 1e-3
batch_size = 32
epochs = 5

In [12]:
torch.manual_seed(42)
# data = DataLoader(background, batch_size=batch_size, shuffle=True)
X_train_dataloader = ...
X_test_dataloader = ...
X_valid_dataloader = ...

In [13]:
model = AutoencoderAnomalyDetection().to(device)
print(model)
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

AutoencoderAnomalyDetection(
  (encoder): Sequential(
    (0): Flatten(start_dim=128, end_dim=256)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=67200, bias=True)
    (7): Unflatten(dim=1, unflattened_size=[3, 32, 700])
  )
  (decoder): Sequential(
    (0): Flatten(start_dim=128, end_dim=256)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=67200, bias=True)
    (7): Unflatten(dim=1, unflattened_size=[3, 32, 700])
  )
)


### Training Model